# Imports

In [2]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np

In [21]:
session = requests.session()
session.max_redirects = 60

# empty df as dict to append values
result_df = {'Pack': [], 'Song Name': [], 'Artist': [], 'Difficulty': [], 'Level': [],
             'Chart Constant': [], 'Notes': [], 'Chart Design': [], 'BPM': [], 'Side': [],
             'Length': [], 'Version': [], 'Vocals': [], 'Genre': [], 'Unlock Requirements': []}

# read and get page data
page = session.get('https://arcaea.fandom.com/wiki/Songs_by_Date')
soup = BeautifulSoup(str(BeautifulSoup(page.content, 'html.parser').findAll('table')), 'html.parser')

# filter and mend song names
song_name_urls = [str(name) for name in soup.findAll("a") if 'href="/wiki/' in str(name)]
song_name_urls = [name.replace('<a href="/wiki/', '').split('" title')[0] for name in song_name_urls]
song_name_urls = [name for name in song_name_urls if not any([name.endswith(img_ext) for img_ext in ['.png', '.jpg']])]
song_name_urls = np.unique([name for name in song_name_urls if '<a' not in name]).tolist()

# read and get page data
page = session.get('https://arcaea.fandom.com/wiki/Songs_by_Date')
soup = BeautifulSoup(page.content, 'html.parser')

# parse into dataframe
tables = soup.findAll("table")
df = pd.concat([pd.read_html(str(table))[0] for table in tables])

# combine 'Length' and 'Len'
df['Length'].fillna(df['Len'], inplace=True)
df.drop('Len', axis=1, inplace=True)

# iterate over song names as displayed in url
print(len(song_name_urls))
for song_name in song_name_urls:
    print(song_name)

    # get song data


    # read and get page data
    page = session.get(f'https://arcaea.fandom.com/wiki/{song_name}')
    soup = BeautifulSoup(page.content, 'html.parser')
    
    # grab first 3 tables
    page_dfs = soup.findAll('table')
    page_dfs = [pd.read_html(str(page_df))[0] for page_df in page_dfs]
    
    # check table columns
    try:
        page_dfs[0][['Level', 'Notes', 'Chart Constant']]
        page_dfs[1][['Background', 'Chart Design', 'BPM']]
        page_dfs[2][['Side', 'Artwork', 'Length']]
        page_dfs[3][['Added', 'Vocals', 'Genre']]

    except (KeyError, IndexError):
        continue
    
    break


360
%CE%91terl%CE%B2us


In [159]:
session = requests.session()
session.max_redirects = 60

# empty df as dict to append values
result_df = {'Pack': [], 'Song Name': [], 'Artist': [], 'Difficulty': [], 'Level': [],
             'Chart Constant': [], 'Notes': [], 'Chart Design': [], 'BPM': [], 'Side': [],
             'Length': [], 'Version': [], 'Vocals': [], 'Genre': [], 'Unlock Requirements': []}

# read and get page data
page = session.get('https://arcaea.fandom.com/wiki/Songs_by_Date')
soup = BeautifulSoup(page.content, 'html.parser')

# parse into dataframe
tables = soup.findAll("table")
df = pd.concat([pd.read_html(str(table))[0] for table in tables])

# combine 'Length' and 'Len'
df['Length'].fillna(df['Len'], inplace=True)

# manually correct song 'World.execute(me);' at pos 150, name doesn't match url
df.at[150, 'Song'] = 'World.execute(me);'

# track failures
failure_ids = []

# iterate over song names
for i, song_name in enumerate(df['Song']):
    try:
        # print(f'\r[{i + 1: >{len(str(len(df["Song"])))}}/{len(df["Song"])}] https://arcaea.fandom.com/wiki/{song_name}' + '\t' * 10, end='')

        # get song row as series
        song_data = df.loc[df['Song'] == song_name].iloc[0]

        # read and get song page data
        if 'Hikari' in song_name:
            song_name += '_(Song)'
        page = session.get(rf'https://arcaea.fandom.com/wiki/{song_name}')
        soup = BeautifulSoup(page.content, 'html.parser')

        if page.status_code == 404:
            failure_ids.append(i)
            print(song_name)
            continue

        # get dataframes
        tables = soup.findAll('table')
        page_dfs = [pd.read_html(str(table))[0] for table in tables]

        # get row count
        difficulty_cols = ['PST', 'PRS', 'FTR', 'BYD']
        difficulty_rows = song_data[difficulty_cols][song_data[difficulty_cols].notna()]
        row_count = len(difficulty_rows)

        # build general data
        result_df['Pack'] += [song_data['Pack']] * row_count
        result_df['Song Name'] += [song_data['Song']] * row_count
        result_df['Artist'] += [song_data['Artist']] * row_count
        result_df['Difficulty'] += difficulty_rows.index.tolist()
        result_df['BPM'] += [song_data['BPM']] * row_count
        result_df['Length'] += [song_data['Length']] * row_count
        result_df['Version'] += [song_data['Update']] * row_count

        # build song page data
        result_df['Level'] += [v for v in page_dfs[0]['Level'].iloc[0].split(' ')]
        result_df['Chart Constant'] += [v for v in page_dfs[0]['Chart Constant'].iloc[0].split(' ')]
        result_df['Notes'] += [v for v in page_dfs[0]['Notes'].iloc[0].split(' ')]
        result_df['Chart Design'] += [v for v in page_dfs[1]['Chart Design'].iloc[0].split(' ')]
        result_df['Side'] += [page_dfs[2]['Side'].iloc[0]] * row_count
        result_df['Vocals'] += [page_dfs[3]['Vocals'].iloc[0]] * row_count
        result_df['Genre'] += [page_dfs[3]['Genre'].iloc[0]] * row_count

        # get unlock requirements
        soup = BeautifulSoup(page.content, 'html.parser')
        unlock_reqs = soup.findAll('span', {'style': 'color:#a0a0a0'})[1:]
        unlock_reqs = [str(req).replace('</span>', '').replace('<span style="color:#a0a0a0">', '') for req in unlock_reqs]

        result_df['Unlock Requirements'] += unlock_reqs

    except Exception as e:
        print(song_name)
        print(e)

PRAGMATISMPRAGMATISM -RESURRECTION-
IgnotusIgnotus Afterburn
Red and BlueRed and Blue and Green
SingularitySingularity VVVIP
dropdeadoverdead.
Solitary Dream
'numpy.float64' object has no attribute 'split'
AI[UE]OON
#1f1e33
'Level'
Alice á la mode
Can I Friend You on Bassbook? Lol
LastLast | MomentLast | Eternity
[X]
Unlisted
Tutorial
'Level'
April Fools' Day


In [160]:
failure_ids

[17, 40, 53, 75, 92, 132, 182, 207, 282, 305, 323, 325]

In [148]:
for k, v in result_df.items():
    print(k, len(v))

Pack 991
Song Name 991
Artist 991
Difficulty 991
Level 951
Chart Constant 951
Notes 1015
Chart Design 634
BPM 991
Side 979
Length 991
Version 991
Vocals 979
Genre 979
Unlock Requirements 540


In [149]:
result_df['Unlock Requirements']

['None',
 'None',
 'None',
 'None',
 'None',
 'None',
 'None',
 'None',
 'None',
 'None',
 'None',
 'None',
 'None',
 'None',
 'None',
 'None',
 'None',
 'None',
 'None',
 'None',
 'None',
 'None',
 'None',
 'None',
 'None',
 'None',
 'None',
 'None',
 'None',
 'None',
 'None',
 'None',
 'None',
 'None',
 'None',
 'None',
 'None',
 'None',
 'None',
 'None',
 'None',
 'None',
 'None',
 'None',
 'None',
 'None',
 'None',
 'None',
 'None',
 'None',
 'None',
 'None',
 'None',
 'None',
 'None',
 'None',
 'None',
 'None',
 'None',
 'None',
 'None',
 'None',
 'None',
 'None',
 'None',
 'None',
 'None',
 'None',
 'None',
 'None',
 'None',
 'None',
 'None',
 'None',
 'None',
 'None',
 'None',
 'None',
 'None',
 'None',
 'None',
 'None',
 'None',
 'None',
 'None',
 'None',
 'None',
 'None',
 'None',
 'None',
 'None',
 'None',
 'None',
 'None',
 'None',
 'None',
 'None',
 'None',
 'None',
 'None',
 'None',
 'None',
 'None',
 'None',
 'None',
 'None',
 'None',
 'None',
 'None',
 'None',
 'None',
 

In [161]:
failure_ids

[17, 40, 53, 75, 92, 132, 182, 207, 282, 305, 323, 325]

In [162]:
df.iloc[17]

Image                                     NaN
Song      PRAGMATISMPRAGMATISM -RESURRECTION-
Artist                                   Laur
PST                                         4
PRS                                         8
FTR                                        10
BYD                                        11
Len                                  2:002:37
BPM                                       174
Pack                                       EC
Update                     1.0.8 3.10.0 [BYD]
Length                               2:002:37
Name: 17, dtype: object